### Import libs

In [1]:
import pandas as pd
import numpy as np
import re
import os
import glob
import json
import csv

### For removing logs errors and warnings

In [2]:
import logging

log = logging.getLogger()
log.setLevel('DEBUG')
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s"))
log.addHandler(handler)

In [3]:
import logging
logging.getLogger('cassandra').setLevel(logging.ERROR)

### Creating list of filepaths to process original event csv data files

In [4]:
print(os.getcwd())

/home/omar/BigData/Data Modeling with apache cassandra/Data Modeling With Apache Cassandra


In [5]:
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
full_data_rows_list = []   # initiating an empty list of rows that will be generated from each file
for file in file_path_list:
    with open(file,'r',encoding='utf8',newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) # for skipping header
        for row in reader:
            full_data_rows_list.append(row)  # extracting each data row one by one and append it  

In [7]:
df = pd.DataFrame(full_data_rows_list)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.54094E+12,648,Everlasting,200,1.54241E+12,49
1,Randy Crawford,Logged In,Chloe,F,54,Cuevas,251.402,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.54094E+12,648,Rio De Janeiro Blue (Album Version),200,1.54241E+12,49
2,Placebo,Logged In,Chloe,F,55,Cuevas,224.02567,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.54094E+12,648,Breathe Underwater,200,1.54241E+12,49
3,Poison The Well,Logged In,Chloe,F,56,Cuevas,184.60689,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.54094E+12,648,Riverside,200,1.54241E+12,49
4,Justin Bieber,Logged In,Chloe,F,57,Cuevas,196.88444,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.54094E+12,648,U Smile,200,1.54241E+12,49


### Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables


In [8]:
csv.register_dialect('dialect',quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile.csv', 'w', encoding='utf8', newline='') as file:
    writer = csv.writer(file, dialect='dialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if(row[0]==''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [9]:
# checking the number of rows in your csv file
with open('event_datafile.csv','r',encoding='utf8') as file:
    print(sum(1 for line in file))

6821


## Apache Cassandra database connection

!pip install cassandra-driver

In [10]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])  # This should make a connection to instance your local machine (127.0.0.1)
# To establish connection and begin executing queries, need a session
session = cluster.connect()

### Creating Keyspace called sparkify

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

In [ ]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### First Query:  Selecting artist name , song title and song's length for a certain session ID and item in sessions number.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS music_lib_by_session
           (session_id int,item_in_session int, artist text,song text, length float , PRIMARY KEY (session_id,item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = "event_datafile.csv"
with open (file, 'r', encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # for skipping header
    
    for line in csvreader:
        query = """INSERT INTO music_lib_by_session (session_id, item_in_session, artist, song, length)
        VALUES (%s, %s, %s, %s, %s)"""
        session.execute(query,(int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Doing a SELECT to verify that the data have been inserted into each table

In [ ]:
query = "SELECT * FROM sparkify.music_lib_by_session WHERE session_id = 648 AND item_in_session = 55;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
for row in rows:
    print('Artist: {0}\nsessionId: {1}\nItemInSession: {2}\nLength: {3}\nSong: {4}\n'\
          .format(row.artist,row.session_id,row.item_in_session,row.length,row.song))

In [ ]:
q1 = pd.DataFrame(data = rows,columns=['Artist','sessionId','ItemInSession','Length','Song'])
q1

### Insight:
**The expected output should be 'Faithless' as the artist and the song was 'Music Matters (Mark Knight Dub)' with a length of 495.307312 seconds.**

### Second Query: Selecting artist name , song title (sorted by item in sessions number) and user's first and second name for a certain user ID and Session ID.

In [ ]:
try:
    session.execute("""
                    CREATE TABLE IF NOT EXISTS music_lib_by_user (
                    user_id INT,
                    session_id INT,
                    item_in_session INT,
                    artist TEXT,
                    song TEXT,
                    first_name TEXT,
                    last_name TEXT,
                    PRIMARY KEY ((user_id,session_id),item_in_session))
    """)
except Exception as e:
    print(e)

In [ ]:
file = "event_datafile.csv"
with open(file, 'r',encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # for skipping header
    
    for line in csvreader:
        query = """
        INSERT INTO music_lib_by_user (user_id, session_id, item_in_session, artist, song, first_name, last_name)
        VALUES (%s,%s,%s,%s,%s,%s,%s)
        """
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [ ]:
query = "SELECT artist,song,first_name,last_name FROM music_lib_by_user where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
for row in rows:
    print('Artist: {0}\nsong: {1}\nfirst_name: {2}\nlast_name:'\
          .format(row.artist,row.song,row.first_name,row.last_name))

In [ ]:
q2 = pd.DataFrame(data = rows,columns=['Artist','Song','first_name','last_name'])
q2

### Insight:
**The expected output should be that Sylvie Cruz listened to 4 songs: 'Keep on Keepin' On' by 'Down To The Bone', 'Greece 2000' by 'Three Drives', 'Kilometer' by 'Sebastien Tellier', and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' by 'Lonnie Gordon'.**

### Third Query: Selecting every user's name that listened to a certain song.

In [ ]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_info_by_song(
        song text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song,user_id))
    """)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO user_info_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
rows = session.execute("SELECT song, first_name,last_name FROM user_info_by_song where song='All Hands Against His Own'")

In [ ]:
for row in rows:
    print(row.first_name,row.last_name)

In [ ]:
q3 = pd.DataFrame(data = rows,columns=['song','first_name','last_name'])
q3

### Insight:
**The expected output should be three users listened to the song 'All Hands Against His Own': Jacqueline Lynch, Sara Johnson, and Tegan Levine.**